# Crowd-YOLO (CYOLO)
- Step 0: Prepare necessary environment
- Step 1: Prepare data
- Step 2: Run experiments

### Step 0: Prepare necessary environment
- Ensure that there exists "master" data:
  - Images (`<data_directory>/images/<image_name>.jpg`)
  - Labels (`<data_directory>/labels/<volunteer_name>/<image_name>.txt`)
    - Experts would give ground-truth labels
    - Non-experts would give crowdsourced labels
- From the master data, we prepare multiple datasets for multiple experiments based on the following modes:
  - Size (**sz**) modes: toy (**to**), full (**fu**)
    - When in the toy mode, we only use a small sample of images (and their annotations). This is done mainly to check that our code works well. It's only a unit-test for our code. But the main experiment would be conducted on the full data (in the full (**fu**) mode).
  - Multiplicity (**ml**) modes: single (**si**), all (**all**)
    - To use labels from a single (**si**) labeller or multiple (**all**) labellers.
  - Train-Validation-Test modes:
    - Train (**tr**), Validation (**va**), Test (**te**)
      - What data to be used for training, validation, and testing of the model learnt.
    - Experts (**e**), Non-experts (**ne**)
      - Since labellers could either be experts or non-experts, we need to decide whether we want to use labels from experts or from non-experts. See below for examples.
    - Repeated (**r**), Crowdsourced (**c**)
      - If an algorithm that accepts training data in the usual form (i.e., one set of labels per image), we resolve "multiple" labellers' labels by repeating the (image, labels) combination as many number of times as there are labellers. More clearly, if we have a dataset as follows:
        - $(x_1, (y_{11}, y_{12}, \cdots, y_{1k_1}))$
        - $(x_2, (y_{21}, y_{22}, \cdots, y_{2k_2}))$
        - $\vdots$
        - $(x_n, (y_{n1}, y_{n2}, \cdots, y_{nk_n}))$,
      
        having $n$ images, with the $i^\text{th}$ image $x_i$ having labelled by $k_i$ labellers giving labels $y_{i1}, y_{i2}, \cdots, y_{ik_i}$. To use YOLO, a usual object detection algorithm, we transform this data via the "repeated" (**r**) mode into $\sum_{i=1}^{n}{k_i}$ (data, label) combinations:
        - $(x_1, y_{11})$
        - $(x_1, y_{12})$
        - $\cdots$
        - $(x_1, y_{1k_1})$,
        - $(x_2, y_{21})$
        - $(x_2, y_{22})$
        - $\cdots$
        - $(x_2, y_{2k_2})$,
        - $\vdots$
        - $(x_n, y_{n1})$
        - $(x_n, y_{n2})$
        - $\cdots$
        - $(x_n, y_{nk_n})$.
        
      - To be able to make use of the crowdsourced labels fully, we use Crowd-YOLO, and this is when we go with the "crowdsourced" (**c**) mode.
- For example, when running YOLO (without BCC), we have the following mode settings:
  - Size mode: full (**sz=fu**)
  - Multiplicity mode: multiple (**ml=all**)
  - Train-Validation-Test mode:
    - Train on non-expert repeated (iid) data: **tr=ner**
    - Validation on non-expert repeated (iid) data: **va=ner**
    - Test on expert repeated (iid) data: **te=er**
- But when running CYOLO (i.e., Crowd-YOLO), we have the following mode settings:
  - Size mode: full (**sz=fu**)
  - Multiplicity mode: multiple (**ml=all**)
  - Train-Validation-Test mode:
    - Train on non-expert crowdsourced data: **tr=nec**
    - Validation on non-expert repeated (iid) data: **va=ner**
    - Test on expert repeated (iid) data: **te=er**
- The modes that we use:
  - `singletoy-yolo`: **sz_to.ml_si.tr_ner.va_ner.te_er** --> For a quick run on the toy data using labels from a single volunteer for YOLO.
  - `singlefull-yolo`: **sz_fu.ml_si.tr_ner.va_ner.te_er** --> For the full data using labels from a single volunteer for YOLO.
  - `alltoy-yolo`: **sz_to.ml_all.tr_ner.va_ner.te_er** --> For a quick run on the toy data using all volunteers' labels for YOLO.
  - `allfull-yolo`: **sz_fu.ml_all.tr_ner.va_ner.te_er** --> For the full data using all volunteers' labels for YOLO.
  - `singletoy-cyolo`: **sz_to.ml_si.tr_nec.va_ner.te_er** --> For a quick run on the toy data using labels from a single volunteer for CYOLO.
  - `singlefull-cyolo`: **sz_fu.ml_si.tr_nec.va_ner.te_er** --> For the full data using labels from a single volunteer for CYOLO.
  - `multipletoy-cyolo`: **sz_to.ml_all.tr_nec.va_ner.te_er** --> For a quick run on the toy data using all volunteers' labels for CYOLO.
  - `multiplefull-cyolo`: **sz_fu.ml_all.tr_nec.va_ner.te_er** --> For the full data using all volunteers' labels for CYOLO.

### Step 1: Prepare data
- 

In [24]:
def get_data_mode_textual_name(data_mode):
    w1 = 'single' if data_mode['ml'] == 'si' else 'all'
    w2 = 'toy' if data_mode['sz'] == 'to' else 'full'
    w3 = 'yolo' if data_mode['tr'] == 'ner' else 'cyolo'
    assert data_mode['tr'] in ['ner', 'nec'] and \
           data_mode['va'] == 'ner' and \
           data_mode['te'] == 'er', "Undefined textual name for this data mode."
    textual_name = f'{w1}{w2}_{w3}'
    return textual_name
    
def get_data_mode_technical_name(data_mode):
    key_seq = ['sz', 'ml', 'tr', 'va', 'te']
    technical_name = '.'.join([f'{x}_{data_mode[x]}' for x in key_seq])
    return technical_name

In [25]:
SINGLETOY_YOLO_MODE =   {'sz': 'to', 'ml': 'si',  'tr': 'ner', 'va': 'ner', 'te': 'er'}
SINGLEFULL_YOLO_MODE =  {'sz': 'fu', 'ml': 'si',  'tr': 'ner', 'va': 'ner', 'te': 'er'}
ALLTOY_YOLO_MODE =      {'sz': 'to', 'ml': 'all', 'tr': 'ner', 'va': 'ner', 'te': 'er'}
ALLFULL_YOLO_MODE =     {'sz': 'fu', 'ml': 'all', 'tr': 'ner', 'va': 'ner', 'te': 'er'}

SINGLETOY_CYOLO_MODE =  {'sz': 'to', 'ml': 'si',  'tr': 'nec', 'va': 'ner', 'te': 'er'}
SINGLEFULL_CYOLO_MODE = {'sz': 'fu', 'ml': 'si',  'tr': 'nec', 'va': 'ner', 'te': 'er'}
ALLTOY_CYOLO_MODE =     {'sz': 'to', 'ml': 'all', 'tr': 'nec', 'va': 'ner', 'te': 'er'}
ALLFULL_CYOLO_MODE =    {'sz': 'fu', 'ml': 'all', 'tr': 'nec', 'va': 'ner', 'te': 'er'}

In [26]:
data_modes = [SINGLETOY_YOLO_MODE, SINGLETOY_CYOLO_MODE]

In [27]:
[get_data_mode_technical_name(x) for x in data_modes]

['sz_to.ml_si.tr_ner.va_ner.te_er', 'sz_to.ml_si.tr_nec.va_ner.te_er']

In [28]:
[get_data_mode_textual_name(x) for x in data_modes]

['singletoy_yolo', 'singletoy_cyolo']